In [1]:
import os 
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY']=os.getenv('OPENAI_API_KEY')

In [2]:
# Langsmith Tracking
os.environ['LANGCHAIN_API_KEY']=os.getenv('LANGCHAIN_API_KEY')
os.environ['LANGCHAIN_TRACING_V2']="true"
os.environ['LANGCHAIN_PROJECT']="GenAIProject"

# Data Ingestion

## PDF

In [3]:
from pathlib import Path
from langchain_community.document_loaders import PyPDFLoader

project_root = Path().resolve().parent
pdf_dir = project_root / "data" / "raw" / "pdf"

all_docs = []

pdf_files = [p for p in pdf_dir.iterdir() if p.is_file() and p.suffix.lower() == ".pdf"]

for pdf in pdf_files:
    loader = PyPDFLoader(str(pdf))
    docs = loader.load()

    for doc in docs:
        doc.metadata["source_type"] = "pdf"
        doc.metadata["source_name"] = pdf.name
        doc.metadata["source_id"] = pdf.name

    all_docs.extend(docs)

incorrect startxref pointer(1)
parsing for Object Streams
found 0 objects within Object(174,0) whereas 100 expected
found 0 objects within Object(175,0) whereas 100 expected
found 0 objects within Object(176,0) whereas 100 expected
found 0 objects within Object(177,0) whereas 100 expected
found 0 objects within Object(178,0) whereas 100 expected
found 0 objects within Object(179,0) whereas 100 expected
found 0 objects within Object(180,0) whereas 100 expected
found 0 objects within Object(181,0) whereas 100 expected
found 0 objects within Object(182,0) whereas 100 expected
found 0 objects within Object(183,0) whereas 100 expected
found 0 objects within Object(184,0) whereas 100 expected
found 0 objects within Object(185,0) whereas 100 expected
found 0 objects within Object(186,0) whereas 100 expected
found 0 objects within Object(187,0) whereas 100 expected
found 0 objects within Object(188,0) whereas 10 expected


In [4]:
all_docs

[Document(metadata={'source': 'C:\\Users\\nico_\\Desktop\\GenAIProject\\data\\raw\\pdf\\20241127-bro-guide-ragv4-interactif.pdf', 'page': 0, 'source_type': 'pdf', 'source_name': '20241127-bro-guide-ragv4-interactif.pdf', 'source_id': '20241127-bro-guide-ragv4-interactif.pdf'}, page_content=''),
 Document(metadata={'source': 'C:\\Users\\nico_\\Desktop\\GenAIProject\\data\\raw\\pdf\\20241127-bro-guide-ragv4-interactif.pdf', 'page': 1, 'source_type': 'pdf', 'source_name': '20241127-bro-guide-ragv4-interactif.pdf', 'source_id': '20241127-bro-guide-ragv4-interactif.pdf'}, page_content=''),
 Document(metadata={'source': 'C:\\Users\\nico_\\Desktop\\GenAIProject\\data\\raw\\pdf\\20241127-bro-guide-ragv4-interactif.pdf', 'page': 2, 'source_type': 'pdf', 'source_name': '20241127-bro-guide-ragv4-interactif.pdf', 'source_id': '20241127-bro-guide-ragv4-interactif.pdf'}, page_content=''),
 Document(metadata={'source': 'C:\\Users\\nico_\\Desktop\\GenAIProject\\data\\raw\\pdf\\20241127-bro-guide-ragv4

In [5]:
# Nettoyage des documents
import re

def basic_clean(text: str) -> str:
    if not text:
        return ""
    text = text.replace("\x00", " ")
    text = re.sub(r"\s+", " ", text)
    return text.strip()

pdf_docs = []
for d in all_docs:
    d.page_content = basic_clean(d.page_content)
    if len(d.page_content) > 80:   # enlève pages vides / quasi vides
        pdf_docs.append(d)

In [6]:
len(pdf_docs)

624

In [7]:
pdf_docs[0].metadata

{'source': 'C:\\Users\\nico_\\Desktop\\GenAIProject\\data\\raw\\pdf\\a-practical-guide-to-building-agents.pdf',
 'page': 1,
 'source_type': 'pdf',
 'source_name': 'a-practical-guide-to-building-agents.pdf',
 'source_id': 'a-practical-guide-to-building-agents.pdf'}

## Pages Web

In [8]:
from langchain_community.document_loaders import WebBaseLoader
import bs4

urls = [
    "https://docs.cloud.google.com/vertex-ai/generative-ai/docs/learn/overview?utm_source=chatgpt.com&hl=fr",
    "https://en.wikipedia.org/wiki/Retrieval-augmented_generation?utm_source=chatgpt.com",
    "https://en.wikipedia.org/wiki/Prompt_engineering?utm_source=chatgpt.com",
    "https://fr.wikipedia.org/wiki/Recherche_g%C3%A9n%C3%A9rative_assist%C3%A9e_par_intelligence_artificielle?utm_source=chatgpt.com",
    "https://github.com/aishwaryanr/awesome-generative-ai-guide?utm_source=chatgpt.com"
]

loader = WebBaseLoader(web_paths=urls)
web_docs=loader.load()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [9]:
len(web_docs)

5

In [10]:
web_docs[0].metadata

{'source': 'https://docs.cloud.google.com/vertex-ai/generative-ai/docs/learn/overview?utm_source=chatgpt.com&hl=fr',
 'title': "Guide du débutant sur l'IA générative \xa0|\xa0 Generative AI on Vertex AI \xa0|\xa0 Google Cloud Documentation",
 'description': "Découvrez les workflows d'IA générative dans Vertex\xa0AI, les modèles disponibles (y compris Gemini) et comment commencer à créer votre application d'IA générative.",
 'language': 'fr-x-mtfrom-en'}

In [11]:
for d in web_docs:
    d.metadata["source_type"] = "web"
    d.metadata["source_name"] = d.metadata.get("title", "web")
    d.metadata["source_id"] = d.metadata.get("source")
    d.metadata.pop("source", None)

In [12]:
web_docs[0].metadata

{'title': "Guide du débutant sur l'IA générative \xa0|\xa0 Generative AI on Vertex AI \xa0|\xa0 Google Cloud Documentation",
 'description': "Découvrez les workflows d'IA générative dans Vertex\xa0AI, les modèles disponibles (y compris Gemini) et comment commencer à créer votre application d'IA générative.",
 'language': 'fr-x-mtfrom-en',
 'source_type': 'web',
 'source_name': "Guide du débutant sur l'IA générative \xa0|\xa0 Generative AI on Vertex AI \xa0|\xa0 Google Cloud Documentation",
 'source_id': 'https://docs.cloud.google.com/vertex-ai/generative-ai/docs/learn/overview?utm_source=chatgpt.com&hl=fr'}

## Fusion PDF et Web

In [13]:
all_documents = pdf_docs + web_docs
len(all_documents)

629

In [14]:
all_documents

[Document(metadata={'source': 'C:\\Users\\nico_\\Desktop\\GenAIProject\\data\\raw\\pdf\\a-practical-guide-to-building-agents.pdf', 'page': 1, 'source_type': 'pdf', 'source_name': 'a-practical-guide-to-building-agents.pdf', 'source_id': 'a-practical-guide-to-building-agents.pdf'}, page_content='C o n t e n t s Wha t is an agen t? 4 When should y ou build an agen t? 5 A gen t design f ounda tions 7 Guar dr ails 2 4 Conclusion 32 2 P r a c t i c a l g u i d e t o b u i l d i n g a g e n t s'),
 Document(metadata={'source': 'C:\\Users\\nico_\\Desktop\\GenAIProject\\data\\raw\\pdf\\a-practical-guide-to-building-agents.pdf', 'page': 2, 'source_type': 'pdf', 'source_name': 'a-practical-guide-to-building-agents.pdf', 'source_id': 'a-practical-guide-to-building-agents.pdf'}, page_content='I n t r o d u c t i o n L ar ge language models ar e becoming incr easingly capable o f handling comple x, multi-st ep task s. A dv ances in r easoning, multimodality , and t ool use ha v e unlock ed a ne w ca

# Data transformation

In [15]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=150,
    separators=["\n\n", "\n", ".", " ", ""]
)

chunks = text_splitter.split_documents(all_documents)


In [16]:
len(chunks)

1295

In [17]:
print(chunks[100].page_content[:500])
print(chunks[100].metadata)


. Ils sont encore plus nombreux à s’inquiéter des risques liés à la fuite de données, la sécurité et la réputation. En outre, la plupart des entreprises financent dans une certaine mesure les recherches actuelles sur l’IA générative, mais de nombreux dirigeants (30 %) ne sont pas convaincus de son impact significatif sur leur secteur dans les deux prochaines années. Pour la plupart, la question n’est pas de savoir si l’IA générative aura un impact sur leur entreprise, mais plutôt de combien de t
{'source': 'C:\\Users\\nico_\\Desktop\\GenAIProject\\data\\raw\\pdf\\gen-ai-handbook-fr.pdf', 'page': 2, 'source_type': 'pdf', 'source_name': 'gen-ai-handbook-fr.pdf', 'source_id': 'gen-ai-handbook-fr.pdf'}


# Embedding

In [18]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small") # changer zen large mais augmentation du coût

# Vector Stores

In [19]:
from langchain_community.vectorstores import Chroma

vectorstore=Chroma.from_documents(documents=chunks,embedding=embeddings,persist_directory="./chroma_db",collection_name="genai_docs") # collection = un index vectoriel logique dans Chroma

vectorstore.persist() # sauvegarde la base vectorielle sur le disque, permet de ne pas les garder qu'en mémoire car au redémarrage du notebook tout est perdu

C:\Users\nico_\AppData\Local\Temp\ipykernel_29916\3859373857.py:5: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist() # sauvegarde la base vectorielle sur le disque, permet de ne pas les garder qu'en mémoire car au redémarrage du notebook tout est perdu


# Test du retriever

In [ ]:
retriever = vectorstore.as_retriever(
    search_type="mmr",  # MMR = Max Marginal Relevance, l'objectif par rapport à "similarity" c'est de garder des chunks pertinents mais différents entre eux, pour éviter la redondance
                        # similarité cosinus + pénalisation de la redondance
    search_kwargs={
        "k": 5, # 5 chunks diversifiés
        "fetch_k": 20 # Combien récupère t'on de chunks et plus fetch_k est grand, plus MMR a de choix et plus la diversité est bonne
    }
)

query = "Qu'est ce que le RAG?"

docs_retrieved = retriever.invoke(query)

for d in docs_retrieved:
    print(d.metadata["source_type"], d.metadata["source_name"])
    print(d.page_content[:200])
    print("---")

web Retrieval-augmented generation - Wikipedia
The term RAG was first introduced in a 2020 research paper.[3]
---
pdf rag-driven-generative-ai.pdf
. The RAG framework can be summed up at a high level in the following figure:
---
pdf rag-driven-generative-ai.pdf
RAG Store: A **RAG (Retrieval-Augmented Generation) vector store specialized type of database or dataset that is designed to store vectorized data points… Naïve RAG can be sufficient in many situation
---
pdf rag-driven-generative-ai.pdf
Naïve RAG: This type of RAG framework doesn’t involve complex data embedding and indexing. It can be efficient to access reasonable amounts of data through keywords, for example, to augment a user’s i
---
web Guide du débutant sur l'IA générative  |  Generative AI on Vertex AI  |  Google Cloud Documentation
Optimiser les requêtes
AperçuOptimiseur zero-shotOptimiseur basé sur les donnéesUtiliser des modèles de requêtes

Moteur RAG
Présentation de RAGGuide de démarrage rapide RAGFacturation du mot

# Test chaîne RAG simple

In [23]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4.1-mini",
    temperature=0
)

In [26]:
from langchain_core.prompts import ChatPromptTemplate

rag_prompt = ChatPromptTemplate.from_messages(
    [
        ("system",
         "You are an expert AI Engineer."
         "Answer the question using only the provided context."
         "If the answer is not in the cintzxt, say 'I don't know'."
         ),
         (
             "user",
             "CONTEXT:\n{context}\n\nQUESTION:\n{question}"
         )
    ]
)

In [27]:
chain = rag_prompt | llm

In [31]:
def rag_answer(question:str):
    docs = retriever.invoke(question) # retriever interroge chroma, il retourne une liste de documents
                                      # Chaque document contient : page_content (texte) et metadata (source, pdf/web)

    context = "\n\n".join(d.page_content for d in docs) # Concaténation brute des chunks récupérés, séparés par des sauts de ligne

    print("=== CONTEXT ===")
    print(context[:1500])
    print("===============")


    response = chain.invoke( # Appel de la chain
        {
            "context":context,
            "question":question # rag_prompt reçoit le context et la question. Un prompt final est généré et est envoyé au llm OpenAI
        }
    )

    return response.content

In [32]:
rag_answer("Can you explain what LangSmith is?")

=== CONTEXT ===
Agents

Building RAG Agents with LLMs by Nvidia
Functions, Tools and Agents with LangChain by Deeplearning.AI
AI Agents in LangGraph by Deeplearning.AI
AI Agentic Design Patterns with AutoGen by Deeplearning.AI
Multi AI Agent Systems with crewAI by Deeplearning.AI
Building Agentic RAG with LlamaIndex by Deeplearning.AI
LLM Observability: Agents, Tools, and Chains by Arize AI
Building Agentic RAG with LlamaIndex by Deeplearning.AI
Agents Tools & Function Calling with Amazon Bedrock (How-to) by AWS Developers
ChatGPT & Zapier: Agentic AI for Everyone by Coursera
Multi-Agent Systems with AutoGen by Victor Dibia [Book]
Large Language Model Agents MOOC, Fall 2024 by Dawn Song & Xinyun Chen – A comprehensive course covering foundational and advanced topics on LLM agents.
CS294/194-196 Large Language Model Agents by UC Berkeley

Miscellaneous

Avoiding AI Harm by Coursera
Developing AI Policy by Coursera


📎 Resources

ICLR 2024 Paper Summaries


💻 Interview Prep
Topic wise Qu

"LangSmith is a tool used for the storage and evaluation of datasets, particularly in the context of evaluating language models' ability to interact with tools and handle specific tasks end-to-end, as exemplified in the Langchain Benchmark."

In [33]:
rag_answer("Qu'est ce qu'un RAG?")

=== CONTEXT ===
The term RAG was first introduced in a 2020 research paper.[3]

RAG Store: A **RAG (Retrieval-Augmented Generation) vector store specialized type of database or dataset that is designed to store vectorized data points… Naïve RAG can be sufficient in many situations. However, if the volume of documents becomes too large or the content becomes more complex, then advanced RAG configurations will provide better results. Let’s now explore advanced RAG. 3. Advanced RAG As datasets grow larger, keyword search methods might prove too long to run. For instance, if we have hundreds of documents and each document contains hundreds of sentences, it will become challenging to use keyword search only. Using an index will reduce the computational load to just a fraction of the total data. In this section, we will go beyond searching text with keywords. We will see how RAG transforms text data into numerical representations, enhancing search efficiency and processing speed

. The RAG f

"Un RAG (Retrieval-Augmented Generation) est un cadre qui améliore les grands modèles de langage (LLM) en incorporant une étape de recherche d'information avant de générer des réponses. Contrairement aux LLM qui s'appuient uniquement sur des données d'entraînement statiques, le RAG extrait des textes pertinents à partir de bases de données, de documents téléchargés ou de sources web. Cela permet de réduire les hallucinations de l'IA, d'éviter la nécessité de réentraîner les modèles avec de nouvelles données, et d'inclure des sources dans les réponses pour plus de transparence et de vérification. Il existe plusieurs types de RAG : naïf (recherche par mots-clés simple), avancé (recherche vectorielle et indexée pour gérer de grands volumes de données complexes), et modulaire (intégrant RAG naïf, avancé, apprentissage automatique et autres algorithmes pour des projets complexes)."

In [34]:
rag_answer("Qu'est ce que la genai?")

=== CONTEXT ===
Restez organisé à l'aide des collections
    

      
      Enregistrez et classez les contenus selon vos préférences.
    









Ce guide pour débutants vous présente les technologies de base de l'IA générative et explique comment elles s'articulent pour alimenter les chatbots et les applications.
L'IA générative (également appelée genAI ou gen AI) est un domaine du machine learning (ML) qui développe et utilise des modèles de ML pour générer de nouveaux contenus.
Les modèles d'IA générative sont souvent appelés grands modèles de langage (LLM) en raison de leur grande taille et de leur capacité à comprendre et à générer du langage naturel.
Toutefois, selon les données sur lesquelles les modèles sont entraînés, ils peuvent comprendre et générer du contenu à partir de plusieurs modalités, y compris du texte, des images, des vidéos et de l'audio. Les modèles qui fonctionnent avec plusieurs modalités de données sont appelés modèles multimodaux.

L’IA générative va trans

"La genAI, ou IA générative, est un domaine du machine learning qui développe et utilise des modèles pour générer de nouveaux contenus. Ces modèles, souvent appelés grands modèles de langage (LLM), sont capables de comprendre et de générer du langage naturel, ainsi que du contenu à partir de plusieurs modalités comme le texte, les images, les vidéos et l'audio."